In [12]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt

# Import the API key
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")

In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
snowfall = input("Do you want it to be snowing? (yes/no) ")
rainfall = input("Do you want it to be raining? (yes/no) ")

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90
Do you want it to be snowing? (yes/no) no
Do you want it to be raining? (yes/no) no


In [4]:
# Filter df for max and min temp matching cities
preferred_cities_df_temp = city_data_df[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 

In [5]:
# Filter df for snowfall and rain choice matching cities

if snowfall == "no" and rainfall == "no":
    preferred_cities = preferred_cities_df_temp[(preferred_cities_df_temp['Snowfall inches (last 3 hrs)']==0) & 
                                                (preferred_cities_df_temp['Rain inches (last 3 hrs)']==0)]
elif snowfall == "yes" and rainfall == "yes":
        preferred_cities = preferred_cities_df_temp[(preferred_cities_df_temp['Snowfall inches (last 3 hrs)']>0) | 
                                                (preferred_cities_df_temp['Rain inches (last 3 hrs)']>0)]
    
elif snowfall == "yes" and rainfall == "no":
        preferred_cities = preferred_cities_df_temp[(preferred_cities_df_temp['Snowfall inches (last 3 hrs)']>0) & 
                                                (preferred_cities_df_temp['Rain inches (last 3 hrs)']==0)]
    
else:
        preferred_cities = preferred_cities_df_temp[(preferred_cities_df_temp['Snowfall inches (last 3 hrs)']==0) & 
                                                (preferred_cities_df_temp['Rain inches (last 3 hrs)']>0)]
    

In [6]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df['Hotel Name'] = ""

In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# GET HOTEL NAMES

# Base URL for getting hotelnames from google api
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
  
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
hotel_df.to_csv("weather_data/WeatherPy_vacation.csv")
